In [1]:
from algorithm import GraphWrapper
from pathlib import Path
from tqdm import tqdm

input_path = Path('../data/benign_graphs/tc3-theia/firefox/nd')
input_paths = list(input_path.glob('*.json'))[:200]
input_graphs = [GraphWrapper(input_path) 
                for input_path in tqdm(input_paths, desc='Reading graphs')]
len(input_graphs)

Reading graphs: 100%|██████████| 200/200 [00:08<00:00, 24.42it/s]


200

In [2]:
from collections import deque
from algorithm import GraphWrapper, EdgeWrapper, NodeWrapper, Subgraph, IN, OUT

def get_subgraphs(graph: GraphWrapper, direction: str) -> list[Subgraph]:
    result: list[Subgraph] = []
    visited_edges: set[EdgeWrapper] = set()

    queue = deque([(0, graph.source_edge_id)])
    while len(queue) > 0:
        # Pop
        depth, edge_id = queue.popleft()
        edge: EdgeWrapper = graph.get_edge(edge_id)
        if edge in visited_edges:
            continue
        visited_edges.add(edge)
        
        # Add subgraph
        result.append(Subgraph(
            parent_graph=graph,
            depth=depth,
            edges=graph.get_subtree(edge_id, direction)
        ))
        
        # Extend queue
        node_id: int = edge.node_ids[direction]
        node: NodeWrapper = graph.get_node(node_id)
        next_edge_ids: list[int] = node.edge_ids[direction]

        queue.extend([(depth + 1, next_edge_id)
                      for next_edge_id in next_edge_ids])
    return result

subgraphs = []
for graph in input_graphs:
    subgraphs.extend(get_subgraphs(graph, IN))
len(subgraphs)

196148

In [3]:
x = [graph.depth for graph in subgraphs]
min(x), max(x)

(0, 6)

In [5]:
from algorithm.utility import to_nx
nx_graphs = [to_nx(graph) 
             for graph in tqdm(subgraphs, desc='Converting to NetworkX graphs')]
len(nx_graphs)

Converting to NetworkX graphs: 100%|██████████| 196148/196148 [00:11<00:00, 17747.90it/s]


196148

In [6]:
from karateclub import Graph2Vec

graph2vec = Graph2Vec(
    wl_iterations=80,
    attributed=True,
    dimensions=128,
    workers=4,
    epochs=5
)

graph2vec.fit(nx_graphs)

graph2vec_embedding = graph2vec.get_embedding()
len(graph2vec_embedding)

196148

# I. Graph2Vec Embedding Analysis

## A. Embedding Variance
The embedding is not deterministic - it varies every time it is computed. 
TODO: look into how the graph2vec.embeddings() works.
compare embedding from graph2vec.embeddings() with graph2vec.infer(graph) - mean .97ish, std around .02 


In [84]:
import random

import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

indices = list(range(len(nx_graphs)))
sample_indices = random.choices(indices, k=20000)
columns = ['index', 'embedding_i', 'inferred_i', 'cosine_similarity']
df = pd.DataFrame(columns=columns)

for i in tqdm(sample_indices):
    graph = subgraphs[i]
    row = {
        'index': i,
        'depth': graph.depth,
        '#edges': len(graph.edges),
        '#nodes': len(graph.nodes),
        'embedding_i': graph2vec_embedding[i],
        'inferred_i': graph2vec.infer([nx_graphs[i]])[0],
        'cosine_similarity': cosine_similarity([graph2vec_embedding[i]], [graph2vec.infer([nx_graphs[i]])[0]])[0, 0]
    }
    df = df.append([row], ignore_index=True)

 34%|███▍      | 33872/100000 [03:10<06:10, 178.26it/s]


KeyboardInterrupt: 

In [ ]:
df['#edges'] = df['index'].apply(lambda i: len(subgraphs[i].edges))

In [83]:
# group df by #edges
grouped_data = df.groupby('#edges')['cosine_similarity'].agg(['mean', 'std', 'count'])
grouped_data

,mean,std,count
#edges,,,
1.0,0.971427,0.020596,9878
2.0,0.976955,0.023588,20
3.0,0.956525,0.024739,5
4.0,0.974760,0.003769,5
5.0,0.963782,NaN,1
7.0,0.979989,0.005538,4
17.0,0.987531,NaN,1
22.0,0.966198,NaN,1
25.0,0.980197,NaN,1


In [53]:
np.std(similarities)

0.020014646

In [ ]:
import pickle
with open('graph2vec_embedding.p', 'wb') as f:
    pickle.dump(graph2vec_embedding, f)
with open('subgraphs.p', 'wb') as f:
    pickle.dump(subgraphs, f)

In [20]:
from sklearn.metrics.pairwise import cosine_similarity
from itertools import combinations

data = {
    'delta_depth': [],
    'depth_1': [],
    'depth_2': [],
    'cosine_similarity': [],
    'parent_graph': []
}

graph_ids = list(set([s.graph.source_edge_id for s in subgraphs]))[:5]
for i_g, graph_id in enumerate(graph_ids):
    contained_subgraphs = [(i, s) for i, s in enumerate(subgraphs) if s.graph.source_edge_id == graph_id]
    comparisons = list(combinations(contained_subgraphs, 2))
    for (i, s1), (j, s2) in tqdm(comparisons, desc=f'{i_g+1}/{len(graph_ids)} ({graph_id})'):
        data['depth_1'].append(s1.depth)
        data['depth_2'].append(s2.depth)
        data['cosine_similarity'].append(cosine_similarity([graph2vec_embedding[i]], [graph2vec_embedding[j]])[0][0])


4/5: 100%|██████████| 500500/500500 [01:23<00:00, 5969.79it/s]


In [21]:
import pandas as pd
df = pd.DataFrame(data)
df.head()

ValueError: All arrays must be of the same length

In [89]:

grouped_data = df.groupby('delta_depth')['cosine_similarity'].agg(['mean', 'std'])
grouped_data.head()

KeyError: 'delta_depth'

In [88]:
import networkx as nx
G = nx_graphs[0]
# This displays the graph as a png
nx.draw(G, with_labels=True)

TypeError: '_AxesStack' object is not callable

<Figure size 640x480 with 0 Axes>

## Similarity between paths

In [44]:
import random
from sklearn.metrics.pairwise import cosine_similarity
from algorithm.utility import to_nx

g = input_graphs[0]
paths: list[list[EdgeWrapper]] = []
nx_path_graphs = []
def generate_path(edge_id: int, direction: str) -> list[EdgeWrapper]:
    visited: dict[int, bool] = {}
    edges: list[EdgeWrapper] = [g.get_edge(g.source_edge_id)]
    current = g.get_edge(edge_id)
    
    # very ugly loop, it works though... #TODO: refactor
    while current is not None:
        visited[current.get_ref_id()] = True
        edges.append(current)
        node_id = current.node_ids[direction]
        node = g.get_node(node_id)
    
        next_edge_ids = node.edge_ids[direction]
        if len(next_edge_ids) == 0:
            break
        next_edge_id = random.choice(node.edge_ids[direction])
        current = g.get_edge(next_edge_id)
    return edges

def generate_path_subgraphs(graph: GraphWrapper, direction: str, n: int) -> list[Subgraph]:
    path_edges: list[list[EdgeWrapper]] = []
    for _ in range(n):
        path_edges.append(generate_path(graph.source_edge_id, direction))
    return [ Subgraph(parent_graph=graph, edges=edges)
             for edges in path_edges ]

nx_path_subgraphs = []

for graph in tqdm(input_graphs):
    nx_path_subgraphs.extend(generate_path_subgraphs(graph, IN, 10000))
    
len(nx_path_subgraphs)

100%|██████████| 200/200 [00:08<00:00, 24.47it/s]


2000000

In [43]:
len(input_graphs)

200

In [45]:
lengths = [len(x.edges) for x in nx_path_subgraphs ]
min(lengths), max(lengths)

(1, 12)

In [48]:
import numpy as np
np.std(lengths)

0.14399361138258995

In [53]:
candidates = [g for g in nx_path_subgraphs if len(g.edges) > 6]
len(candidates)

9

In [73]:
import networkx as nx
import matplotlib.pyplot as plt
G = to_nx(candidates[0])
# Draw the graph
fig, ax = plt.subplots()  # Create a figure and an axes object
nx.draw(G,  ax=ax, with_labels=True, node_color='lightblue', node_size=500, font_size=10, font_weight='bold')
# TODO: PICK UP HERE - I want to make sure this is actually a path...
# Draw the nodes and edges
pos = nx.spring_layout(G)
nx.draw(G, pos,ax=ax, with_labels=True, node_color='lightblue', node_size=500, font_size=10, font_weight='bold')

# Draw edge labels
edge_labels = nx.get_edge_attributes(G, 'weight')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, ax=ax)

# Show the plot
plt.show()

In [ ]:
for edge in 